In [1]:
import math
import numpy as np
from Value import Value
from MLP import *
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# loading dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X / 255.0 # normalize pixels
y = y.astype(int)

In [3]:
# splitting data
np.random.seed(42)
indices = np.random.permutation(len(X))
split = int(0.8 * len(X))
train_idx, test_idx = indices[:split], indices[split:]
X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

np.shape(X_train)

(56000, 784)

In [4]:
# ie. 4 = [0,0,0,0,1,0,0,0,0,0]
def encode_labels(y):
    one_hot = np.zeros((len(y), 10))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot
y_train_encoded = encode_labels(y_train)
y_test_encoded = encode_labels(y_test)

In [5]:
# Model that will be used
model = MLP(196, [64, 32, 10])

In [6]:
# using different loss function
def softmax(logits):
    exps = [x.exp() for x in logits]
    sum_exps = sum(exps, Value(0))
    probs = [e / sum_exps for e in exps]
    return probs

def cross_entropy_loss(logits, true_label):
    probs = softmax(logits)
    loss = Value(0)
    for p, t in zip(probs, true_label):
        if t == 1:
            loss -= p.log()
    return loss

In [7]:
def update_weights(model, base_lr, e, decay=0.96):
    lr = base_lr * (decay ** e)
    for p in model.parameters():
         p.data += -lr * p.grad

In [8]:
from PIL import Image
import numpy as np

def resize_dataset(X, new_size=(14,14)):
    resized_images = []
    for img_array in X:
        img = Image.fromarray(img_array)
        img_small = img.resize(new_size, Image.Resampling.LANCZOS)
        resized_images.append(np.array(img_small)) 
    return np.array(resized_images)

In [9]:
# the training begins
X_train = resize_dataset(X_train, (14, 14))
X_test = resize_dataset(X_test, (14, 14))
X_train = X_train[:1000]
y_train_encoded = y_train_encoded[:1000]
epochs = 25
for epoch in range(epochs):
    
    total_loss = 0
    for x, y_true in zip(X_train, y_train_encoded):
        # forward pass
        x_val = [Value(xi) for xi in np.ravel(x)]
        y_pred = model(x_val)
        loss = cross_entropy_loss(y_pred, y_true)

        # zero gradients
        model.zero_grad()

        # backward pass
        loss.backward()

        # update weights
        update_weights(model, base_lr=0.2, e=epoch)
        total_loss += loss.data

    print(f"Epoch: {epoch + 1}, Loss: {total_loss / len((X_train))}")

Epoch: 1, Loss: 2.2454805343543205
Epoch: 2, Loss: 2.0232444748754665
Epoch: 3, Loss: 1.7693693496854408
Epoch: 4, Loss: 1.7799512900885743
Epoch: 5, Loss: 1.7285176814106158
Epoch: 6, Loss: 1.7502250647151925
Epoch: 7, Loss: 1.6498328222736915
Epoch: 8, Loss: 1.6203661782618106
Epoch: 9, Loss: 1.6000600966102032
Epoch: 10, Loss: 1.5859353882264737
Epoch: 11, Loss: 1.5449178988880528
Epoch: 12, Loss: 1.5765309994075867
Epoch: 13, Loss: 1.498767041947442
Epoch: 14, Loss: 1.505951049846193
Epoch: 15, Loss: 1.4792164467840363
Epoch: 16, Loss: 1.461747570380834
Epoch: 17, Loss: 1.4866997657120464
Epoch: 18, Loss: 1.4880395745628714
Epoch: 19, Loss: 1.4295851511780222
Epoch: 20, Loss: 1.4430259377564132
Epoch: 21, Loss: 1.4283028523018284
Epoch: 22, Loss: 1.4236325692250325
Epoch: 23, Loss: 1.3975621678404415
Epoch: 24, Loss: 1.3729332112476222
Epoch: 25, Loss: 1.3571698259395797
